In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_absolute_error


In [2]:
# Batch of inputs (game states, one-hot encoded)
X = tf.placeholder(tf.float32, [None, 13, 26, 9], name="X") # type tf.float32 is needed for the rest of operations

# Batch of outputs (correct predictions of number of actions)
Y_corr = tf.placeholder(tf.float32, [None, 1], name="Y")

In [3]:
# Neural Network Architecture

# Flatten input
X_flat = tf.layers.flatten(X)

# First Layer - Receives inputs
layer_1 = tf.layers.dense(inputs = X_flat,
                                  units = 512,
                                  activation = tf.nn.relu,
                                  kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                  use_bias = True,
                                  name="layer_1")

with tf.variable_scope("layer_1", reuse=True): # Get weights and bias for histogram
    weights_1 = tf.get_variable("kernel")
    bias_1 = tf.get_variable("bias")
    
# Second Layer
layer_2 = tf.layers.dense(inputs = layer_1,
                                  units = 32,
                                  activation = tf.nn.relu,
                                  kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                  use_bias = True,
                                  name="layer_2")

# Output Layer
output = tf.layers.dense(inputs = layer_2,
                                  units = 1,
                                  activation = None,
                                  kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                  use_bias = True,
                                  name="output")

with tf.variable_scope("output", reuse=True): # Get weights and bias for histogram
    weights_output = tf.get_variable("kernel")
    bias_output = tf.get_variable("bias")

In [4]:
# Training
loss = tf.reduce_mean(tf.square(output - Y_corr), name="loss") # Quadratic loss

"""learning_rate = tf.train.exponential_decay(learning_rate=0.0008,
                                          global_step= 1,
                                          decay_steps=5000,
                                          decay_rate= 0.95,
                                          staircase=True)""" # Funciona peor

alfa = tf.placeholder(tf.float64, name='alfa') # Mejor valor alfa=0.001

optimizer = tf.train.AdamOptimizer(learning_rate=alfa, name="optimizer")
train_op = optimizer.minimize(loss, name="train_op")

In [5]:
# TensorBoard

# Summaries
tf.summary.scalar('loss', loss)
tf.summary.histogram("weigths_1", weights_1)
tf.summary.histogram("bias_1", bias_1)
tf.summary.histogram("weigths_output", weights_output)
tf.summary.histogram("bias_output", bias_output)

merged_summary = tf.summary.merge_all() # Operación para obtener todos los valores de los summaries

In [6]:
# Session

# Load Dataset
dataset = np.load('datasets/dataset_train_3.npz')
dataset_x = dataset['X']
dataset_y = dataset['Y']

# dataset_x.fill(1) # QUITAR DESPUÉS
# dataset_y.fill(20)

# Number of epochs
num_epochs = 80
# Minibatch size
batch_size = 128

# Learning rates to test
alfas = [0.001]

for this_alfa in alfas:

    with tf.Session() as sess:
        writer = tf.summary.FileWriter("ModeloPlanificacion_log/prueba_2capas_4")
        writer.add_graph(tf.get_default_graph())
        
        sess.run(tf.global_variables_initializer()) # Initialize all variables

        rkf = RepeatedKFold(n_splits=int(len(dataset_y) // batch_size),
                      n_repeats=num_epochs) # Get randomized indexes for minibatches

        it = 0

        for _, batch_index in rkf.split(dataset_y):
            batch_x = np.take(dataset_x, batch_index, axis=0) # Obtain current batch
            batch_y = np.take(dataset_y, batch_index)
            batch_y = np.reshape(batch_y, (-1, 1))

            data_dict = {X:batch_x, Y_corr:batch_y, alfa:0.002}

            summary = sess.run(merged_summary, feed_dict=data_dict) # Get Summaries
            writer.add_summary(summary, it) # Write it to log in disk

            sess.run(train_op, feed_dict=data_dict) # Execute one training step

            it += 1


        # --- After training ---
        # Compute MAE
            
        dataset_y_reshaped = np.reshape(dataset_y, (-1, 1))   
        data_dict = {X:dataset_x, Y_corr:dataset_y_reshaped}
        
        y_pred = sess.run(output, feed_dict=data_dict)
        
        mae_base = mean_absolute_error(dataset_y, np.repeat(np.mean(dataset_y), dataset_y.shape[0]))
        mae_model = mean_absolute_error(dataset_y, y_pred)
        
        print("\n\n---MAE---\n\n")
        print("Model:", mae_model)
        print("Baseline:", mae_base)
        
        #mae_summary = tf.summary.scalar('MAE', MAE) # Summaries
        #model_mae = sess.run(mae_summary, feed_dict=data_dict)
        #writer.add_summary(model_mae)
            

        # Now predict some values (OF TRAINING SET -> CAN BE OVERFITTED!)

        """
        print("\n\nPREDICTIONS\n\n")
        rkf = RepeatedKFold(n_splits=int(len(dataset_y) // batch_size),
                          n_repeats=1) # Get randomized indexes for minibatches

        for _, batch_index in rkf.split(dataset_y):
            batch_x = np.take(dataset_x, batch_index, axis=0) # Obtain current batch
            batch_y = np.take(dataset_y, batch_index)
            batch_y = np.reshape(batch_y, (-1, 1))

            data_dict = {X:batch_x, Y_corr:batch_y, alfa:this_alfa}

            pred = sess.run(output, feed_dict=data_dict) # Get Predictions for current batch

            print("Predictions:", pred)
            print("Real values:", batch_y)
            print("\n\n")"""
    



---MAE---


Model: 1.123654884869468
Baseline: 6.7887650132224895
